In [1]:
import torch
torch.cuda.is_available()

C:\Users\DELL\anaconda3\envs\ta\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
from datasets import load_dataset
from datasets import Dataset
dataset = load_dataset("SetFit/subj")
dataset = dataset.remove_columns("label_text")
dataset = dataset['train'].shuffle(seed=42).select([i for i in list(range(6000))])
train_ds = Dataset.from_dict(dataset[0:5000])
validation_ds = Dataset.from_dict(dataset[5000:5500])
test_ds = Dataset.from_dict(dataset[5500:6000])





Using custom data configuration SetFit___subj-848ec40cb3887b72
Reusing dataset json (C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426)
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 83.19it/s]
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\json\SetFit___subj-848ec40cb3887b72\0.0.0\c2d554c3377ea79c7664b93dc65d0803b45e3279000f993c7bfd18937fd7f426\cache-80b8e4129e527f71.arrow


In [3]:
display(train_ds)
display(validation_ds)
display(test_ds)

Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

Dataset({
    features: ['text', 'label'],
    num_rows: 500
})

In [4]:
display(train_ds[0:5])

{'text': ["on this morning , johnny wakes to his mother 's accusations that he has botched a simple scam , leaving them without enough money to payoff their inside man .",
  "despite a quieter middle section , involving aragorn 's dreams of arwen , this is even better than the fellowship . there are scenes of cinematic perfection that steal your heart away .",
  '[ kidd ] can write dialogue , work skillfully with actors , and he has a pretty good handle on urban loneliness of the knowing , virulent new york city variety .',
  'abandoned and filled with a mysterious past , holy angel school for girls is about to reveal its secrets of betrayal , jealousy , and vengeance .',
  "the romantic triangle leads to a surprising conclusion as the young man 's secret motive has explosive consequences ."],
 'label': [0, 1, 1, 0, 0]}

In [5]:
display(validation_ds[0:5])

{'text': ["he has all the kit but none of the skill , and he 's a laughing stock at school .",
  'although disney follows its standard formula in this animated adventure , it feels more forced than usual .',
  'several uninteresting , unlikeable people do bad things to and with each other in `` unfaithful . `` why anyone who is not a character in this movie should care is beyond me .',
  "'you 'll laugh for not quite and hour and a half , but come out feeling strangely unsatisfied . you 'll feel like you ate a reeses without the peanut butter . . . '",
  'a forceful drama of an alienated executive who re-invents himself .'],
 'label': [0, 1, 1, 1, 1]}

In [6]:
display(test_ds[0:5])

{'text': ['he can remember nothing and begins to try to rebuild his memory based on clues such as the swiss bank account , the number of which , is implanted in his hip .',
  'leaving her successful career and las vegas forever in order to build her family in a healthy environment was since long prevented by katherine hiller ( barbara carrera ) , her discoverer and business manager .',
  "a film worthy of comparison to truffaut 's best cinematic poems .",
  'in the end there is one word that best describes this film : honest .',
  'angry and confused teen runaways come and go at the shelter , as steve learns he is not alone in his predicament and that the streets offer a harsh reality .'],
 'label': [0, 0, 1, 1, 0]}

In [7]:
from transformers import set_seed

set_seed(42)

In [8]:
#defining model:
from transformers import (AutoTokenizer, 
                          AutoModelForSequenceClassification, 
                          Trainer, EarlyStoppingCallback,
                          TrainingArguments)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenize_func = lambda sentences: tokenizer(sentences['text'], 
                                            padding='max_length', truncation=True, max_length = 100)



train_dataset = train_ds.map(tokenize_func, batched=True)
val_dataset = validation_ds.map(tokenize_func, batched=True)
test_dataset = test_ds.map(tokenize_func, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Parameter 'function'=<function <lambda> at 0x000001B7F2A00280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 52.88ba/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceCla

In [9]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}


training_args = TrainingArguments(
    output_dir=r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model',          # output directory
    num_train_epochs=20,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.1,               # strength of weight decay
    
    logging_dir=r'C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\logs\original_model',            # directory for storing logs
    save_total_limit = 1,
    load_best_model_at_end=True,
    #metric_for_best_model='f1',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    gradient_accumulation_steps=2,
    learning_rate=2e-5,
)


trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [10]:
#training:
trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])
# We are going to get multiple loss values on each training step here


The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 5000
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 2
  Total optimization steps = 1560


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.685400,0.622450,0.890000,0.888447
1,0.382900,0.213245,0.926000,0.925714
2,0.168100,0.170546,0.934000,0.933637
3,0.116200,0.145002,0.952000,0.951870
4,0.071200,0.205265,0.930000,0.929997
5,0.038100,0.199965,0.946000,0.945989
6,0.025800,0.302351,0.930000,0.929986


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\metrics\accuracy\3e9ee15abf476145152fe4e9a9c1463ff95d3d65cdc555be9cfe061bdaeb1a14 (last modified on Sat Aug 13 21:10:49 2022) since it couldn't be found locally at accuracy, or remotely on the Hugging Face Hub.
Saving model checkpoint to C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model\checkpoint-78
Configuration saved in C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model\checkpoint-78\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model\checkpoint-78\pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `Di

TrainOutput(global_step=546, training_loss=0.21253916108127915, metrics={'train_runtime': 280.0657, 'train_samples_per_second': 357.059, 'train_steps_per_second': 5.57, 'total_flos': 905538858000000.0, 'train_loss': 0.21253916108127915, 'epoch': 6.99})

In [11]:
#save model:
trainer.save_state()
trainer.save_model()

Saving model checkpoint to C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model
Configuration saved in C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model\config.json
Model weights saved in C:\Users\DELL\Text_Augmentation\subj_dataset_3runs\models\original_model\pytorch_model.bin


In [12]:
#evaluate:
trainer.evaluate(test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\metrics\f1\6a64529c5745513ceb230ce9696115dcad6ae0fedad9946e3f2f723a65a0cd53 (last modified on Sat Aug 13 21:10:53 2022) since it couldn't be found locally at f1, or remotely on the Hugging Face Hub.


{'eval_loss': 0.1209598183631897,
 'eval_accuracy': 0.958,
 'eval_f1': 0.9579715887940248,
 'eval_runtime': 33.3534,
 'eval_samples_per_second': 14.991,
 'eval_steps_per_second': 0.48,
 'epoch': 6.99}